In [1]:
# Importing necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading Card Transactions Raw Data
data = pd.read_excel('card transactions.xlsx')

## Exclusions

In [3]:
#Retaining only P transaction type
data = data[data['Transtype'] == 'P']
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [4]:
#Removing the largest transaction (It is in Pesos)
data = data[data['Amount'] < 3000000]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96397 non-null  int64         
 1   Cardnum            96397 non-null  int64         
 2   Date               96397 non-null  datetime64[ns]
 3   Merchnum           93199 non-null  object        
 4   Merch description  96397 non-null  object        
 5   Merch state        95377 non-null  object        
 6   Merch zip          92097 non-null  float64       
 7   Transtype          96397 non-null  object        
 8   Amount             96397 non-null  float64       
 9   Fraud              96397 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 8.1+ MB


In [5]:
#% of missing records
data.count() * 100 /len(data)

Recnum               100.000000
Cardnum              100.000000
Date                 100.000000
Merchnum              96.682469
Merch description    100.000000
Merch state           98.941876
Merch zip             95.539280
Transtype            100.000000
Amount               100.000000
Fraud                100.000000
dtype: float64

In [6]:
#checking nulls
data.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3198
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

## Data Cleaning

In [7]:
# Replacing 0s with nan
data['Merchnum'] = data['Merchnum'].replace(np.nan,'0')
data['Merch state'] = data['Merch state'].replace(np.nan,'0')
data['Merch zip'] = data['Merch zip'].replace(np.nan,0)
data.isnull().sum()

Recnum               0
Cardnum              0
Date                 0
Merchnum             0
Merch description    0
Merch state          0
Merch zip            0
Transtype            0
Amount               0
Fraud                0
dtype: int64

### Filling Merchnum

In [8]:
data['Merchnum'].replace({'0':np.nan},inplace = True)
data.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3251
Merch description       0
Merch state             0
Merch zip               0
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [9]:
data['Merchnum']=data.groupby(['Merch description','Merch zip'])['Merchnum'].transform(lambda x:x.fillna(x.mode()[0]) if not x.mode().empty else 'Empty')
data['Merchnum'].replace({'Empty':np.nan},inplace = True)
data['Merchnum']=data.groupby(['Merch description','Merch state'])['Merchnum'].transform(lambda x:x.fillna(x.mode()[0]) if not x.mode().empty else 'Empty')
data['Merchnum'].replace({'Empty':np.nan},inplace = True)
data['Merchnum']=data.groupby('Merch description')['Merchnum'].transform(lambda x:x.fillna(x.mode()[0]) if not x.mode().empty else 'Empty')
data['Merchnum']=np.where(data['Merchnum']=='Empty',data['Recnum']*(-1),data['Merchnum'])
data.isnull().sum()

Recnum               0
Cardnum              0
Date                 0
Merchnum             0
Merch description    0
Merch state          0
Merch zip            0
Transtype            0
Amount               0
Fraud                0
dtype: int64

### Filling State & Zip

In [10]:
data['Merch state'].replace({'0':np.nan},inplace = True)
data['Merch zip'].replace({0:np.nan},inplace = True)
data.isnull().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum                0
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

In [11]:
data['Merch state']=data.groupby('Merchnum')['Merch state'].transform(lambda x:x.fillna(x.mode()[0]) if not x.mode().empty else 'Empty')
data['Merch zip']=data.groupby('Merchnum')['Merch zip'].transform(lambda x:x.fillna(x.mode()[0]) if not x.mode().empty else 'Empty')
data['Merch state']=np.where(data['Merch state']=='Empty',data['Recnum']*(-1),data['Merch state'])
data['Merch zip']=np.where(data['Merch zip']=='Empty',data['Recnum']*(-1),data['Merch zip'])
data.isnull().sum()

Recnum               0
Cardnum              0
Date                 0
Merchnum             0
Merch description    0
Merch state          0
Merch zip            0
Transtype            0
Amount               0
Fraud                0
dtype: int64

In [12]:
data.shape

(96397, 10)

In [13]:
#data.to_csv("Cleaned Data.csv")